In [1]:
from OpenMiChroM.ChromDynamics import MiChroM
import numpy as np
import CustomIntegrator as CI
import simtk.unit as unit
import sys
sys.path.append('../')
import ActivePolymer
from openmmtools.integrators import LangevinIntegrator
# from CustomIntegrator import PersistentBrownianIntegrator, AddActivity, CustomBrownianIntegrator

In [2]:
sim = MiChroM(name='test_int', temperature=1.0, time_step=1e-2,velocity_reinitialize=False)
integrator=CI.CustomBrownianIntegrator(collision_rate=0.1, timestep=2e-3, temperature=120)
# integrator=LangevinIntegrator(temperature=120.0*unit.kelvin, collision_rate=0.1/unit.picosecond, timestep=0.01*unit.picosecond, splitting='V R O R V', constraint_tolerance=1e-08, measure_shadow_work=False, measure_heat=False)
sim.setup(platform="opencl",integrator=integrator )


    ***************************************************************************************     
     **** **** *** *** *** *** *** *** OpenMiChroM-1.0.2 *** *** *** *** *** *** **** ****      

         OpenMiChroM is a Python library for performing chromatin dynamics simulations.         
                            OpenMiChroM uses the OpenMM Python API,                             
                employing the MiChroM (Minimal Chromatin Model) energy function.                
      The chromatin dynamics simulations generate an ensemble of 3D chromosomal structures      
      that are consistent with experimental Hi-C maps, also allows simulations of a single      
                 or multiple chromosome chain using High-Performance Computing                  
                            in different platforms (GPUs and CPUs).                             
         OpenMiChroM documentation is available at https://open-michrom.readthedocs.io          

         OpenMiChroM is desc

In [3]:

sim.saveFolder('output')

Chrom10 = sim.create_springSpiral(ChromSeq='chr10_beads.txt', isRing=False)
sim.loadStructure(Chrom10, center=True)
sim.saveStructure(mode = 'auto')


In [4]:
# act_seq=np.array([np.random.uniform(0,1) for _ in range(sim.N)])
act_seq=np.array([1 if val>sim.N/2 else 0 for val in range(int(sim.N))])
CI.AddActivity(sim, act_seq)


Active Force added to 2712 particles


In [5]:

# sim.addFENEBonds(kfb=30.0)
# sim.addAngles(ka=2.0)
# sim.addRepulsiveSoftCore(Ecut=4.0)

# sim.addTypetoType(mu=3.22, rc = 1.78)
# sim.addIdealChromosome(mu=3.22, rc = 1.78, dinit=3, dend=500)

# sim.addFlatBottomHarmonic( kr=5e-3, n_rad=10.0)


In [6]:


print(sim.system, sim.integrator, sim.platform, sim.properties)


<simtk.openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7f88c18efa80> > <CustomIntegrator.CustomBrownianIntegrator; proxy of <Swig Object of type 'OpenMM::CustomIntegrator *' at 0x7f88e07ef330> > <simtk.openmm.openmm.Platform; proxy of <Swig Object of type 'OpenMM::Platform *' at 0x7f88c18efb10> > {}


In [7]:


# ActivePolymer.runSims(sim, nblocks=100, blocksize=100,check_every=10)
sim.initStorage(filename="traj_chr10")
for _ in range(500):
    sim.runSimBlock(100, increment=True)
    sim.saveStructure()
# sim.saveStructure(filename='chr10_collapse',mode='ndb')
sim.storage[0].close()


Number of exceptions: 0
adding force  ActiveForce 0
Positions... 
 loaded!
potential energy is 66.992464
bl=1 pos[1]=[212.7 -12.7 -0.5] dr=3.47 t=0.0ps kin=29.21 pot=67.05 Rg=147.129 SPS=563 
bl=2 pos[1]=[211.7 -14.2 -4.6] dr=3.54 t=0.0ps kin=58.14 pot=67.06 Rg=147.222 SPS=638 
bl=3 pos[1]=[211.7 -11.2 -3.0] dr=3.53 t=0.0ps kin=87.89 pot=67.08 Rg=147.245 SPS=645 
bl=4 pos[1]=[212.8 -11.2 -2.6] dr=3.51 t=0.0ps kin=113.45 pot=67.09 Rg=147.267 SPS=629 
bl=5 pos[1]=[208.9 -6.7 -3.3] dr=3.57 t=0.0ps kin=136.21 pot=67.09 Rg=147.327 SPS=632 
bl=6 pos[1]=[207.7 -8.4 -0.5] dr=3.65 t=0.0ps kin=159.93 pot=67.09 Rg=147.366 SPS=641 
bl=7 pos[1]=[208.4 -6.3 -0.2] dr=3.65 t=0.0ps kin=186.04 pot=67.01 Rg=147.325 SPS=653 
bl=8 pos[1]=[203.5 -4.5 -0.9] dr=3.59 t=0.0ps kin=207.77 pot=66.94 Rg=147.381 SPS=669 
bl=9 pos[1]=[205.4 -4.1 -2.6] dr=3.67 t=0.0ps kin=233.51 pot=66.90 Rg=147.409 SPS=666 
bl=10 pos[1]=[207.3 -4.1 -3.8] dr=3.74 t=0.0ps kin=255.46 pot=66.82 Rg=147.447 SPS=673 
bl=11 pos[1]=[207.3 -4.

In [ ]:
sim.removeFlatBottomharmonic()
print('saved collapse')
sim.initStorage(filename="traj_chr10")

block = 500
n_blocks = 50000

for _ in range(n_blocks):
    sim.runSimBlock(block, increment=True)
    sim.saveStructure()

sim.storage[0].close()


In [8]:
def _autocorrFFT(x):
        N=len(x)
        F = np.fft.fft(x, n=2*N)  #2*N because of zero-padding
        PSD = F * F.conjugate()
        res = np.fft.ifft(PSD)
        res= (res[:N]).real   #now we have the autocorrelation in convention B
        n=N*np.ones(N)-np.arange(0,N) #divide res(m) by (N-m)
        return res/n #this is the autocorrelation in convention A

def _msd_fft(r):
    #r is an (T,3) ndarray: [time stamps,dof]
    N=len(r)
    D=np.square(r).sum(axis=1)
    D=np.append(D,0)
    S2=sum([ _autocorrFFT(r[:, i]) for i in range(r.shape[1])])
    Q=2*D.sum()
    S1=np.zeros(N)
    for m in range(N):
        Q=Q-D[m-1]-D[N-m]
        S1[m]=Q/(N-m)
    return S1, S2



In [11]:
from OpenMiChroM.CndbTools import cndbTools
cndbT=cndbTools()


In [13]:

traj=cndbT.load('./output/traj_chr10_0.cndb')
xyz=cndbT.xyz(frames=[1, traj.Nframes,1], beadSelection='all')
print(xyz.shape)

(499, 0, 3)


In [ ]:


msd=[]
for p in range(self.N):
    s1,s2 = _msd_fft(self.xyz[:,p,:])
    # Pos_autocorr.append(2*S2/S1)
    msd.append(s1-2*s2)

msd=np.array(msd)
# Pos_autocorr=np.array(Pos_autocorr)

#average over all particles
msd_av=np.mean(msd,axis=0)